# The search for the loss function 
## TLDR: Failed so far to produce very good results. 
## Usefulness: Shows how to create different custom loss functions, and demonstrates how they can use local variables within them. This also shows how one could calculate the gradient (of some sort) with sparce matrices during training. 
## The main reason for sharing is to show how I am trying to get the gradients for each level 1-12. This is towards the bottom of the notebook. Please let my know if you have any hints for how to make a custom objective function properly. 
This notebook is copied from, and uses the features generated by kyakovlev in [this notebook](https://www.kaggle.com/kyakovlev/m5-simple-fe). They are used to make simple models and compare different objective functions. I have just started testing different objective functions and this notebook could serve as a rough template, although it is unpolished for now. 

I also use ragnars asymetric loss function from [this kernal](https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv) as a baseline

I also use [Tomonori's](https://www.kaggle.com/tnmasui) [this amazing dashboard](https://www.kaggle.com/tnmasui/m5-wrmsse-evaluation-dashboard) with a slight modefication, well noted in the code, to display only the 12 level breakdown of the the wrmsse scores for fast display. This dashboard is amazing and everyone should use it to get more info on how models are doing. 

In [ ]:
import numpy as np 
import pandas as pd 
import os, sys, gc, warnings, psutil, random
from scipy.sparse import csr_matrix

warnings.filterwarnings('ignore')

TARGET = 'sales'      # Our Target
END_TEST = 1913      # Last day of the 28 day test period
NUM_ITERATIONS = 1000   ### SET THIS HIGHER IF YOU WANT TO SEE BETTER TESTING
ID_COLS = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import gc

from sklearn import preprocessing
import lightgbm as lgb

from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm

DATA_DIR = '/kaggle/input/m5-forecasting-accuracy/'

class WRMSSEEvaluator_dashboard(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, 
                 calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 'all'  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')]\
                     .columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')]\
                               .columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], 
                                 axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)\
                    [valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns]\
                    .set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index()\
                   .rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left',
                                    on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd'])\
                    .unstack(level=2)['value']\
                    .loc[zip(self.train_df.item_id, self.train_df.store_id), :]\
                    .reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns],
                               weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt) 

    def score(self, valid_preds: Union[pd.DataFrame, 
                                       np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape \
               == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, 
                                       columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], 
                                 valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):

            valid_preds_grp = valid_preds.groupby(group_id)[self.valid_target_columns].sum()
            setattr(self, f'lv{i + 1}_valid_preds', valid_preds_grp)
            
            lv_scores = self.rmsse(valid_preds_grp, i + 1)
            setattr(self, f'lv{i + 1}_scores', lv_scores)
            
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, 
                                  sort=False).prod(axis=1)
            
            all_scores.append(lv_scores.sum())
            
        self.all_scores = all_scores

        return np.mean(all_scores)
    

    
def create_viz_df(df,lv):
    
    df = df.T.reset_index()
    if lv in [6,7,8,9,11,12]:
        df.columns = [i[0] + '_' + i[1] if i != ('index','') \
                      else i[0] for i in df.columns]
    df = df.merge(calendar.loc[:, ['d','date']], how='left', 
                  left_on='index', right_on='d')
    df['date'] = pd.to_datetime(df.date)
    df = df.set_index('date')
    df = df.drop(['index', 'd'], axis=1)
    
    return df

def create_dashboard(evaluator, by_level_only=False, model_name=None):
    
    wrmsses = [np.mean(evaluator.all_scores)] + evaluator.all_scores
    labels = ['Overall'] + [f'Level {i}' for i in range(1, 13)]

    ## WRMSSE by Level
    plt.figure(figsize=(12,5))
    ax = sns.barplot(x=labels, y=wrmsses)
    ax.set(xlabel='', ylabel='WRMSSE')
    
    #######################ALTERATION##########################
    title = 'WRMSSE by Level'
    if model_name: 
        title = f'WRMSSE by Level for {model_name}'
    plt.title(title, fontsize=20, fontweight='bold')
    #######################ALTERATION-COMPLETE##########################

  
    for index, val in enumerate(wrmsses):
        ax.text(index*1, val+.01, round(val,4), color='black', 
                ha="center")
        
    #######################ALTERATION##########################
    if by_level_only:       # stops function early for quick plotting of 
        plt.show()          # for quick plotting of levels
        return
    #######################ALTERATION-COMPLETE##########################

    # configuration array for the charts
    n_rows = [1, 1, 4, 1, 3, 3, 3, 3, 3, 3, 3, 3]
    n_cols = [1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
    width = [7, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
    height = [4, 3, 12, 3, 9, 9, 9, 9, 9, 9, 9, 9]
    
    for i in range(1,13):
        
        scores = getattr(evaluator, f'lv{i}_scores')
        weights = getattr(evaluator, f'lv{i}_weight')
        
        if i > 1 and i < 9:
            if i < 7:
                fig, axs = plt.subplots(1, 2, figsize=(12, 3))
            else:
                fig, axs = plt.subplots(2, 1, figsize=(12, 8))
                
            ## RMSSE plot
            scores.plot.bar(width=.8, ax=axs[0], color='g')
            axs[0].set_title(f"RMSSE", size=14)
            axs[0].set(xlabel='', ylabel='RMSSE')
            if i >= 4:
                axs[0].tick_params(labelsize=8)
            for index, val in enumerate(scores):
                axs[0].text(index*1, val+.01, round(val,4), color='black', 
                            ha="center", fontsize=10 if i == 2 else 8)
            
            ## Weight plot
            weights.plot.bar(width=.8, ax=axs[1])
            axs[1].set_title(f"Weight", size=14)
            axs[1].set(xlabel='', ylabel='Weight')
            if i >= 4:
                axs[1].tick_params(labelsize=8)
            for index, val in enumerate(weights):
                axs[1].text(index*1, val+.01, round(val,2), color='black', 
                            ha="center", fontsize=10 if i == 2 else 8)
                    
            fig.suptitle(f'Level {i}: {evaluator.group_ids[i-1]}', size=24 ,
                         y=1.1, fontweight='bold')
            plt.tight_layout()
            plt.show()

        trn = create_viz_df(getattr(evaluator, f'lv{i}_train_df')\
                            .iloc[:, -28*3:], i)
        val = create_viz_df(getattr(evaluator, f'lv{i}_valid_df'), i)
        pred = create_viz_df(getattr(evaluator, f'lv{i}_valid_preds'), i)

        n_cate = trn.shape[1] if i < 7 else 9

        fig, axs = plt.subplots(n_rows[i-1], n_cols[i-1], 
                                figsize=(width[i-1],height[i-1]))
        if i > 1:
            axs = axs.flatten()

        ## Time series plot
        for k in range(0, n_cate):

            ax = axs[k] if i > 1 else axs

            trn.iloc[:, k].plot(ax=ax, label='train')
            val.iloc[:, k].plot(ax=ax, label='valid')
            pred.iloc[:, k].plot(ax=ax, label='pred')
            ax.set_title(f"{trn.columns[k]}  RMSSE:{scores[k]:.4f}", size=14)
            ax.set(xlabel='', ylabel='sales')
            ax.tick_params(labelsize=8)
            ax.legend(loc='upper left', prop={'size': 10})

        if i == 1 or i >= 9:
            fig.suptitle(f'Level {i}: {evaluator.group_ids[i-1]}', size=24 , 
                         y=1.1, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
sell_prices = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
train_df = train_df.loc[:, :'d_' + str(END_TEST)]

train_fold_df = train_df.iloc[:, :-28]
valid_fold_df = train_fold_df.iloc[:, -28:].copy()
# Instantiate an evaluator for scoring validation periodstarting day 1886
e = WRMSSEEvaluator_dashboard(train_fold_df, valid_fold_df, calendar, sell_prices)

In [ ]:
# Helper functions 
# Lets get the sample submission file also for our functions 
ss = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')

def score_model(evaluator, model, test_x, test_id, boosters, ss):
    """
    Returns a list of scores for each booster in boosters, 
    which is a list of numbers, denoting the iteration of 
    the model.
    
    boosters: list of numbers
    ss: sample_submission file, as to align the data, 
    needed for most public evaluators
    """
    scores = []
    X_test_id = test_id.copy()
    print('scoring...')
    for booster in boosters: 
        
        preds = model.predict(test_x, num_iteration=booster)
        X_test_id['preds'] = preds
        preds = X_test_id.pivot('id', 'd', 'preds').reset_index()
        preds = pd.merge(ss.iloc[:30490]['id'], preds, on='id', how='left').iloc[:, 1:].values
        scores.append(evaluator.score(preds))
        
    return scores


def get_preds(model,test_x, test_id, ss, booster=None):
    X_test_id = test_id.copy()
    if booster: 
        preds = model.predict(test_x, num_iteration=booster)
    else:
        preds = model.predict(test_x)
    X_test_id['preds'] = preds
    preds = X_test_id.pivot('id', 'd', 'preds').reset_index()
    preds = pd.merge(ss.iloc[:30490]['id'], preds, on='id', how='left').iloc[:, 1:].values
    return preds

In [ ]:
########################### Load data
########################### Basic features were created here:
########################### https://www.kaggle.com/kyakovlev/m5-simple-fe
#################################################################################

# Read data
grid_df = pd.concat([pd.read_pickle('../input/m5-simple-fe/grid_part_1.pkl'),
                     pd.read_pickle('../input/m5-simple-fe/grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('../input/m5-simple-fe/grid_part_3.pkl').iloc[:,2:]],
                     axis=1)



# Training d

In [ ]:
# Subsampling
# to make all calculations faster.
# Keep only data after day 1885-90 (90 days of training data)
grid_df = grid_df[(grid_df.d > 1885-90) & (grid_df.d <= END_TEST)]
# Set up test set (day 1886-1913) 
remove_features = ['id','d',TARGET, 'weight', 'scale'] 
features_columns = [col for col in list(grid_df) if col not in remove_features]
test_x = grid_df[grid_df['d']>(END_TEST-28)][features_columns]
# We also need the id column to merge with sample submission so it is
# in the proper order to score with our evaluator
test_id = grid_df[grid_df['d']>(END_TEST-28)][['id', 'd']]

In [ ]:
########################### Baseline model
#################################################################################

# We will need some global VARS for future

SEED = 42             # Our random seed for everything
random.seed(42)     # to make all tests "deterministic"
np.random.seed(SEED)


# Drop some items from "TEST" set part (1914...)
grid_df = grid_df[grid_df['d']<=END_TEST].reset_index(drop=True)

# Features that we want to exclude from training
remove_features = ['id','d',TARGET, 'weight', 'scale'] 

# Our baseline model serves
# to do fast checks of
# changes to our model weights and objective

# We will use LightGBM for our tests
import lightgbm as lgb


lgb_params = {
                    'boosting_type': 'gbdt',         # Standart boosting type
                    'objective': 'regression',       # Standart loss for RMSE
                    'metric': ['rmse'],              # as we will use rmse as metric "proxy"
                    'subsample': 0.8,                # Lets really speed things up
                    'subsample_freq': 1,
                    'learning_rate': 0.05,           # 0.5 is "fast enough" for us
                    'num_leaves': 2**7-1,            # We will need model only for fast check
                    'min_data_in_leaf': 2**8-1,      # So we want it to train faster even with drop in generalization 
                    'feature_fraction': 0.8,
                    'n_estimators': NUM_ITERATIONS,            # We don't want to limit training (you can change 5000 to any big enough number)
#                     'early_stopping_rounds': 30,     # We will stop training almost immediately (if it stops improving) 
                    'seed': SEED,
                    'verbose': -1
                } 

                
# Small function to make fast features tests
# estimator = make_fast_test(grid_df)
# it will return lgb booster for future analisys
def make_fast_test(df, lgb_params, weight_column=None,weight_valid=None, fobj=None):

    features_columns = [col for col in list(df) if col not in remove_features]

    tr_x, tr_y = df[df['d']<=(END_TEST-28)][features_columns], df[df['d']<=(END_TEST-28)][TARGET]
    if weight_column: 
#         tr_weight = df[df['d']<=(END_TEST-28)][weight_column]
        tr_weight = weight_column # now you must put int the column yourself, not just a 'col' name

    vl_x, v_y = df[df['d']>(END_TEST-28)][features_columns], df[df['d']>(END_TEST-28)][TARGET]
    if weight_valid: 
        vl_weight = df[df['d']>(END_TEST-28)][weight_column]
        
    if weight_column: 
        if weight_valid: 
            train_data = lgb.Dataset(tr_x, label=tr_y, weight=tr_weight)
            valid_data = lgb.Dataset(vl_x, label=v_y, weight=vl_weight)
        else: 
            train_data = lgb.Dataset(tr_x, label=tr_y, weight=tr_weight)
            valid_data = lgb.Dataset(vl_x, label=v_y)
    else: 
        train_data = lgb.Dataset(tr_x, label=tr_y)
        valid_data = lgb.Dataset(vl_x, label=v_y)
    
    
    if fobj: 
        estimator = lgb.train(
                            lgb_params,
                            train_data,
                            valid_sets = [train_data,valid_data],
                            verbose_eval = NUM_ITERATIONS//10,
                            fobj=fobj
                        )
    else: 
        estimator = lgb.train(
                                lgb_params,
                                train_data,
                                valid_sets = [train_data,valid_data],
                                verbose_eval = NUM_ITERATIONS//10,
                            )
    
    
    return estimator

# Make baseline model
# baseline_model = make_fast_test(grid_df, lgb_params)
# baseline_model_poisson = make_fast_test(grid_df, lgb_params_poisson)

In [ ]:
# BOOSTERS = [(NUM_ITERATIONS//10)*i for i in range(1, 11)]
# BOOSTERS

In [ ]:
# res_df = pd.DataFrame({'baseline_model': score_model(e, baseline_model, test_x, test_id, BOOSTERS, ss)}, 
#                      index=BOOSTERS)

# res_df.plot(figsize=(15,6))

# Custom loss function

In [ ]:
# Custom loss function 
def WMSSE(preds, train_data):
    labels = train_data.get_label()
    weight = train_data.get_weight()
    loss = weight*((preds - labels)**2)
    grad = 2 * weight * (preds - labels)
    hess = 2 * weight
    return grad, hess

# Custom objective function: getting the gradients 

In [ ]:
# Get a copy of the training data that will go into 
# the lgbm dataset. We will make scaled_weight
# columns for each level, containing the scaled weight of
# the series to which each row belongs to. Initialize 
# the weith matrix with just the id columns
tr_x = grid_df[grid_df['d']<=(END_TEST-28)]
tr_w = tr_x[ID_COLS]

In [ ]:
# Get the weights and scales for all the levels 
wdf = pd.read_csv('../input/weights-and-scales-for-the-past-2-years/weight_scale_1886.csv')
# Get the sqrt of the scale, because I didn't do 
# that part in the previous notebook. Then divide the weight by the 
# sqrt(scale) to get the proper scaled_weight
wdf['scaled_weight'] = wdf.weight/np.sqrt(wdf.scale)
wdf = wdf[[ 'Level_id', 'Agg_Level_1', 'Agg_Level_2','scaled_weight']]

######################### level 1 #######################################
tr_w['level_1_sw'] = wdf.loc[wdf['Level_id'] == 'Level1', 'scaled_weight'][0]

######################### level 2 #######################################
level = wdf[wdf.Level_id == 'Level2']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id']], level[['state_id', 'scaled_weight']],
                               on='state_id', how='left')[['scaled_weight']]
tr_w['level_2_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 3 #######################################
level = wdf[wdf.Level_id == 'Level3']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id']], level[['store_id', 'scaled_weight']],
                               on='store_id', how='left')[['scaled_weight']]
tr_w['level_3_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 4 #######################################
level = wdf[wdf.Level_id == 'Level4']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'cat_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['cat_id']], level[['cat_id', 'scaled_weight']],
                               on='cat_id', how='left')[['scaled_weight']]
tr_w['level_4_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 5 #######################################
level = wdf[wdf.Level_id == 'Level5']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'dept_id', 'Agg_Level_2', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['dept_id']], level[['dept_id', 'scaled_weight']],
                               on='dept_id', how='left')[['scaled_weight']]
tr_w['level_5_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 6 #######################################
level = wdf[wdf.Level_id == 'Level6']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'cat_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'cat_id']], level[['state_id', 'cat_id', 'scaled_weight']],
                               on=['state_id', 'cat_id'], how='left')[['scaled_weight']]
tr_w['level_6_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 7 #######################################
level = wdf[wdf.Level_id == 'Level7']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'dept_id']], level[['state_id', 'dept_id', 'scaled_weight']],
                               on=['state_id', 'dept_id'], how='left')[['scaled_weight']]
tr_w['level_7_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 8 #######################################
level = wdf[wdf.Level_id == 'Level8']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'cat_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id', 'cat_id']], level[['store_id', 'cat_id', 'scaled_weight']],
                               on=['store_id', 'cat_id'], how='left')[['scaled_weight']]
tr_w['level_8_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 9 #######################################
level = wdf[wdf.Level_id == 'Level9']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'store_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['store_id', 'dept_id']], level[['store_id', 'dept_id', 'scaled_weight']],
                               on=['store_id', 'dept_id'], how='left')[['scaled_weight']]
tr_w['level_9_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 10 #######################################
level = wdf[wdf.Level_id == 'Level10']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'item_id', 'dept_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['item_id']], level[['item_id', 'scaled_weight']],
                               on=['item_id'], how='left')[['scaled_weight']]
tr_w['level_10_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 11 #######################################
level = wdf[wdf.Level_id == 'Level11']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'state_id', 'item_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['state_id', 'item_id']], level[['state_id', 'item_id', 'scaled_weight']],
                               on=['state_id', 'item_id'], how='left')[['scaled_weight']]
tr_w['level_11_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight

######################### level 12 #######################################
level = wdf[wdf.Level_id == 'Level12']
# Set the name of the aggregation column to match 
# the column in our id columns
level.columns = ['Level_id', 'item_id', 'store_id', 'scaled_weight']
level_scaled_weight = pd.merge(tr_w[['item_id', 'store_id']], level[['item_id', 'store_id', 'scaled_weight']],
                               on=['item_id', 'store_id'], how='left')[['scaled_weight']]
tr_w['level_12_sw'] = level_scaled_weight['scaled_weight'].values

del level, level_scaled_weight, wdf
gc.collect()

cols = [col for col in tr_w.columns if 'level' in col]
tr_w = tr_w[cols]

# Roll up matrices for gradients
They are commented out because I can't utilize all of them in a custom loss function so far

In [ ]:
# Get a "dummy matix" to roll up level 1 for each day
# roll_1 = csr_matrix(pd.get_dummies(tr_x.d, drop_first=False).values)
# roll_2 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str'),
#                                    drop_first=False).values)

# roll_3 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str'),
#                                    drop_first=False).values)

# roll_4 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.cat_id.astype('str'),
#                                    drop_first=False).values)

# roll_5 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.dept_id.astype('str'),
#                                    drop_first=False).values)

# roll_6 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.cat_id.astype('str') ,
#                                    drop_first=False).values)

# roll_7 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.dept_id.astype('str'),
#                                    drop_first=False).values)

# roll_8 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str') + tr_x.cat_id.astype('str'),
#                                    drop_first=False).values)

# roll_9 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.store_id.astype('str') + tr_x.dept_id.astype('str'),
#                                    drop_first=False).values)

# roll_10 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.item_id.astype('str') + tr_x.dept_id.astype('str'),
#                                    drop_first=False).values)

# roll_11 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str') + tr_x.item_id.astype('str'),
#                                    drop_first=False).values)

# roll_12 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.item_id.astype('str') + tr_x.store_id.astype('str'),
#                                    drop_first=False).values)

In [ ]:

roll_1 = csr_matrix(pd.get_dummies(tr_x.d, drop_first=False).values)
roll_2 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.state_id.astype('str'),
                                   drop_first=False).values)
roll_4 = csr_matrix(pd.get_dummies(tr_x.d.astype('str') + tr_x.cat_id.astype('str'),
                                   drop_first=False).values)

In [ ]:

def get_level_grad(level_roll, level_sw, actuals, preds):
    """Returns the partial gradient. l1 is a sparse matix 
    made from get_dummies with levels """
    # We only need the differences to "roll up",
    # meaning aggregate and sum (i think)
    diff = actuals - preds

    # Reshape the difference vector so that we 
    # can do matix multiplication l1 * diff
    diff = np.reshape(diff,(level_roll.T.shape[1],-1))

    # Execute the matrix multiplication. The result is 
    # the total difference of sales - predicted for 
    # each day. 
    res = level_roll.T * diff

    # For each day, all we need is the sign of 
    # daily difference, so we "squish" positive 
    # and negative values to 1 and -1. Then we 
    # flip the sign, because the partial derivative
    # with respect to preds is negative when 
    # actuals - preds is positive, and visa versa.
    res = np.where(res < 0, 1, -1)
    
    # reshape the result to be contained in one axis
    res = res.reshape(res.shape[0])
    
    # Expand the result into a matrix that will match
    # the size of level_roll for element-wise multiplication
    res = np.tile(res, (level_roll.shape[0],1))
    
    # Element-wise multiplication 
    res = level_roll.multiply(res)
    
    # Squish sparse matrix into one column
    res = res.sum(axis=1)
    
    # Reshape into single row
    res = np.array(res).reshape(1,-1)[0]

    # Res is now a column with 1 or -1 for each row, 
    # indicating the sign of the derivative of the 
    # level_series to which that row belongs. 
    # This is our gradient with respect to this level. 
    level_grad = level_sw * res
    return level_grad

# Testing: 
Everything after this are my first attempts at making custom functions. The only one that seems promissing right now is obj_10, which is the only time I was able to see slight improvement (must change NUM_ITERATIONS at the top to 200 or more to see it improve) over the asymetric loss function from ragnar in [this kernal](https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv)

In [ ]:
def obj(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
#     g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)
#     g4 = get_level_grad(roll_4, tr_w.level_4_sw, actuals, preds)

    
    sign = np.where(diff<0, 1, -1)
    g12 = tr_w.level_12_sw * sign
    
    grad = g12\
#     + g4\
#     + g1
    grad = grad/np.abs(grad).mean()
    hess = np.where(diff>0, 1, 1)

    return grad, hess

In [ ]:
def obj_6(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
    g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)


    grad = np.where(diff < 0, np.where(g1>0,-1 * diff, -1/2 * diff), np.where(g1<0,-1 * diff, -1/2 * diff))
    hess = np.where(diff < 0, np.where(g1>0,1 , 1/2), np.where(g1<0,1 , 1/2))
    return grad, hess

In [ ]:
def obj_7(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
    g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)


    grad = np.where(diff < 0, np.where(g1>0,-1 * diff, -1/5 * diff), np.where(g1<0,-1 * diff, -1/5 * diff))
    hess = np.where(diff < 0, np.where(g1>0,1 , 1/5), np.where(g1<0,1 , 1/5))
    return grad, hess

In [ ]:
def obj_8(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
    g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)



    grad = np.where(diff < 0, np.where(g1>0,-1 * diff, -4/5 * diff), np.where(g1<0,-1 * diff, -4/5 * diff))
    hess = np.where(diff < 0, np.where(g1>0,1 , 4/5), np.where(g1<0,1 , 4/5))
    return grad, hess

In [ ]:
def obj_9(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
    g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)
    g2 = get_level_grad(roll_2, tr_w.level_2_sw, actuals, preds)
    
    g1 += g2

    grad = np.where(diff < 0, np.where(g1>0,-1 * diff, -4/5 * diff), np.where(g1<0,-1 * diff, -4/5 * diff))
    hess = np.where(diff < 0, np.where(g1>0,1 , 4/5), np.where(g1<0,1 , 4/5))
    return grad, hess

In [ ]:
def obj_10(preds, train_data): 
    actuals = train_data.get_label()
    diff = actuals - preds
    
    g1 = get_level_grad(roll_1, tr_w.level_1_sw, actuals, preds)
    g2 = get_level_grad(roll_2, tr_w.level_2_sw, actuals, preds)
    g4 = get_level_grad(roll_4, tr_w.level_4_sw, actuals, preds)

    g1 += g2 + g4


    grad = np.where(diff < 0, np.where(g1>0,-1 * diff, -4/5 * diff), np.where(g1<0,-1 * diff, -4/5 * diff))
    hess = np.where(diff < 0, np.where(g1>0,1 , 4/5), np.where(g1<0,1 , 4/5))
    return grad, hess

In [ ]:
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)

    return grad, hess


In [ ]:
def custom_asymmetric_train_2(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual )
    hess = np.where(residual < 0, 2, 2 )

    return grad, hess

In [ ]:
m_cat = make_fast_test(grid_df, lgb_params, fobj=custom_asymmetric_train)

In [ ]:
m_cat_2 = make_fast_test(grid_df, lgb_params, fobj=custom_asymmetric_train_2)

In [ ]:
m_obj_6 = make_fast_test(grid_df, lgb_params, fobj=obj_6)

In [ ]:
m_obj_7 = make_fast_test(grid_df, lgb_params, fobj=obj_7)

In [ ]:
m_obj_8 = make_fast_test(grid_df, lgb_params, fobj=obj_8)

In [ ]:
m_obj_9 = make_fast_test(grid_df, lgb_params, fobj=obj_9)

In [ ]:
m_obj_10 = make_fast_test(grid_df, lgb_params, fobj=obj_10)

Comparing results 

In [ ]:
_ = e.score(get_preds(m_cat, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_cat_2, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_obj_6, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_obj_7, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_obj_8, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_obj_9, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)

In [ ]:
_ = e.score(get_preds(m_obj_10, test_x, test_id, ss))
create_dashboard(e, by_level_only=True)